In [19]:
import pandas as pd
import holidays
from datetime import timedelta

#### Read in cleaned data after running initial_data_cleaning.ipynb

In [20]:
df = pd.read_csv('Data/cleaned_parking_violations_v1.csv')

/var/folders/_v/kcxy22753vjfn9jc355xsgs00000gn/T/ipykernel_64247/4208363470.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/cleaned_parking_violations_v1.csv')


In [21]:
df.head(2)

,objectid,ticket_number,issuing_agency_code,issuing_agency_name,issuing_agency_short,violation_code,location,plate_state,disposition_code,disposition_type,...,violation_type_desc,issue_datetime,year,month,day,day_of_week,hour,is_weekend,Unnamed: 29,is_fleet_gov_or_rental
0,81930639,260895666,57,METRO POLICE,MTP,P007,708 7TH ST NW,,0,Other,...,NaN,2023-06-11 15:34:00,2023,6,11,Sunday,15,True,UNKNOWN,True
1,81930640,260895670,57,METRO POLICE,MTP,P007,708 7 ST NW,,0,Other,...,NaN,2023-06-11 15:36:00,2023,6,11,Sunday,15,True,UNKNOWN,True


In [22]:
df['is_fleet_gov_or_rental'].value_counts().sort_index()

is_fleet_gov_or_rental
False         11
True     1618914
Name: count, dtype: int64

#### Convert into weekly time series

In [23]:
df[df['issue_datetime'].isna()]

,objectid,ticket_number,issuing_agency_code,issuing_agency_name,issuing_agency_short,violation_code,location,plate_state,disposition_code,disposition_type,...,violation_type_desc,issue_datetime,year,month,day,day_of_week,hour,is_weekend,Unnamed: 29,is_fleet_gov_or_rental


In [24]:
df['issue_datetime'] = pd.to_datetime(df['issue_datetime'], errors='coerce').dt.tz_localize(None)

df['week'] = df['issue_datetime'].dt.to_period('W').apply(lambda r: r.start_time)

weekly_counts = df.groupby('week').size().reset_index(name='num_violations')

weekly_counts['year'] = weekly_counts['week'].dt.year
weekly_counts['month'] = weekly_counts['week'].dt.month

weekly_counts['weekofyear'] = weekly_counts['week'].dt.isocalendar().week

us_holidays = holidays.US(years=weekly_counts['year'].unique())

In [25]:
def has_holiday_in_week(week_start):
    return any((week_start + timedelta(days=i)) in us_holidays for i in range(7))

weekly_counts['is_holiday_week'] = weekly_counts['week'].apply(has_holiday_in_week)

top_agencies = df['issuing_agency_short'].value_counts().head(5).index.tolist()
df['agency_group'] = df['issuing_agency_short'].where(df['issuing_agency_short'].isin(top_agencies), 'OTHER')

agency_weekly = df.groupby(['week', 'agency_group']).size().unstack(fill_value=0).reset_index()

weekly_counts = weekly_counts.merge(agency_weekly, on='week', how='left')

weekly_counts.fillna(0, inplace=True)

weekly_counts.head()

,week,num_violations,year,month,weekofyear,is_holiday_week,DDOT,DPW,MPD-1D,MPD-2D,MPD-3D,OTHER
0,2022-12-26,239,2022,12,52,True,0,189,1,8,8,33
1,2023-01-02,20130,2023,1,1,True,0,19578,52,51,34,415
2,2023-01-09,28508,2023,1,2,False,0,27914,47,73,29,445
3,2023-01-16,23264,2023,1,3,True,0,22667,59,57,55,426
4,2023-01-23,28931,2023,1,4,False,0,28420,64,43,35,369


In [33]:
weekly_counts.sort_values("num_violations").head(5)

,week,num_violations,year,month,weekofyear,is_holiday_week,DDOT,DPW,MPD-1D,MPD-2D,MPD-3D,OTHER
0,2022-12-26,239,2022,12,52,True,0,189,1,8,8,33
36,2023-09-25,465,2023,9,39,False,12,367,2,11,16,57
51,2025-01-06,7802,2025,1,2,False,32,7474,26,31,31,208
50,2024-12-30,7868,2024,12,1,True,16,7545,37,33,32,205
44,2023-11-20,11806,2023,11,47,True,228,11252,46,32,44,204


In [27]:
weekly_counts["is_holiday_week"].value_counts()

is_holiday_week
False    57
True     15
Name: count, dtype: int64

Identify what weeks have 0 parking violations

In [37]:
full_weeks = pd.date_range(start=weekly_counts['week'].min(), 
                           end=weekly_counts['week'].max(), 
                           freq='W-MON')  # Assuming weeks start on Monday

full_weeks_df = pd.DataFrame({'week': full_weeks})

merged = full_weeks_df.merge(weekly_counts[['week', 'num_violations']], on='week', how='left')

zero_violation_weeks = merged[merged['num_violations'].isna()]

zero_violation_weeks.head()

,week,num_violations
36,2023-09-04,NaN
37,2023-09-11,NaN
38,2023-09-18,NaN
53,2024-01-01,NaN
54,2024-01-08,NaN


In [ ]:
zero_violation_weeks.to_csv('Data/weekly_timeseries_parking_violations_v1.csv', index=False)

In [29]:
weekly_counts.to_csv('Data/weekly_timeseries_parking_violations_v1.csv', index=False)

#### Convert into daily time series

In [30]:
df['issue_datetime'] = pd.to_datetime(df['issue_datetime'])

df['date'] = df['issue_datetime'].dt.tz_localize(None).dt.normalize()

daily_counts = df.groupby('date').size().reset_index(name='num_violations')

daily_counts['year'] = daily_counts['date'].dt.year
daily_counts['month'] = daily_counts['date'].dt.month

us_holidays = holidays.US(years=daily_counts['year'].dropna().astype(int).unique())
holiday_dates = pd.Series(list(us_holidays.keys()), dtype="datetime64[ns]")
daily_counts['is_holiday'] = daily_counts['date'].isin(holiday_dates)
daily_counts['is_weekend'] = daily_counts['date'].dt.dayofweek >= 5
daily_counts['day_of_week'] = daily_counts['date'].dt.day_name()

top_agencies = df['issuing_agency_short'].value_counts().head(5).index.tolist()

df['agency_group'] = df['issuing_agency_short'].where(df['issuing_agency_short'].isin(top_agencies), 'OTHER')

agency_daily = df.groupby(['date', 'agency_group']).size().unstack(fill_value=0).reset_index()

daily_counts = daily_counts.merge(agency_daily, on='date', how='left')

daily_counts.fillna(0, inplace=True)

daily_counts.head()

,date,num_violations,year,month,is_holiday,is_weekend,day_of_week,DDOT,DPW,MPD-1D,MPD-2D,MPD-3D,OTHER
0,2023-01-01,239,2023,1,True,True,Sunday,0,189,1,8,8,33
1,2023-01-02,232,2023,1,True,False,Monday,0,171,6,4,2,49
2,2023-01-03,3401,2023,1,False,False,Tuesday,0,3334,1,7,8,51
3,2023-01-04,4990,2023,1,False,False,Wednesday,0,4925,1,2,3,59
4,2023-01-05,5263,2023,1,False,False,Thursday,0,5141,2,7,10,103


In [ ]:
# Dealing with the missing September 2023 data
full_idx = pd.date_range(daily.index.min(), daily.index.max(), freq='D')
daily = daily.reindex(full_idx)

# Time‑based linear interpolation (respects distance between points)
daily['num_violations'] = daily['num_violations'].interpolate(method='time')

# flag the imputed rows so the model knows they were synthetic
daily['was_imputed'] = daily['num_violations'].isna()


In [31]:
daily_counts["is_holiday"].value_counts()

is_holiday
False    470
True      16
Name: count, dtype: int64

Days with 0 violations

In [36]:
full_dates = pd.date_range(start=daily_counts['date'].min(), 
                           end=daily_counts['date'].max(), 
                           freq='D')

full_dates_df = pd.DataFrame({'date': full_dates})

merged = full_dates_df.merge(daily_counts[['date', 'num_violations']], on='date', how='left')

zero_violation_days = merged[merged['num_violations'].isna()]

zero_violation_days.head()

,date,num_violations
243,2023-09-01,NaN
244,2023-09-02,NaN
245,2023-09-03,NaN
246,2023-09-04,NaN
247,2023-09-05,NaN


In [32]:
daily_counts.to_csv('Data/daily_timeseries_parking_violations_v1.csv', index=False)